In [1]:
import gc
import re
import string
import logging
from copy import copy, deepcopy
import os
import underthesea
import numpy as np
import pyspark
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as f
from pyspark.sql.functions import col, udf, lit, greatest, monotonically_increasing_id, concat_ws
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType, StructType, StructField, MapType, BooleanType
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, MinHashLSH
from pyspark.ml.linalg import Vectors, VectorUDT

In [ ]:
spark = SparkSession.builder \
    .appName("test") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

In [12]:
current_dir = os.getcwd()
json_path_1 = os.path.join(current_dir, "bds.json")
json_path_2 = os.path.join(current_dir, "dothi.json")
json_path_3 = os.path.join(current_dir, "nhadatuytin.json")
json_path_4 = os.path.join(current_dir, "cafeland.json")
json = [json_path_1,json_path_2,json_path_3,json_path_4]


In [13]:
df = spark.read.json(json)

In [14]:
df.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- address: struct (nullable = true)
 |    |-- district: string (nullable = true)
 |    |-- full_address: string (nullable = true)
 |    |-- province: string (nullable = true)
 |    |-- ward: string (nullable = true)
 |-- contact_info: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- phone: string (nullable = true)
 |-- description: string (nullable = true)
 |-- estate_type: string (nullable = true)
 |-- extra_infos: struct (nullable = true)
 |    |-- Hướng ban công: string (nullable = true)
 |    |-- Hướng nhà: string (nullable = true)
 |    |-- Mặt tiền: string (nullable = true)
 |    |-- Pháp lý: string (nullable = true)
 |    |-- Số phòng: string (nullable = true)
 |    |-- Số phòng khách: string (nullable = true)
 |    |-- Số phòng ngủ: string (nullable = true)
 |    |-- Số phòng ngủ :: string (nullable = true)
 |    |-- Số toilet: string (nullable = true)
 |    |-- Số toilet :: string (nullable = tru

In [15]:
df = df.drop("_corrupt_record")
df.printSchema()

root
 |-- address: struct (nullable = true)
 |    |-- district: string (nullable = true)
 |    |-- full_address: string (nullable = true)
 |    |-- province: string (nullable = true)
 |    |-- ward: string (nullable = true)
 |-- contact_info: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- phone: string (nullable = true)
 |-- description: string (nullable = true)
 |-- estate_type: string (nullable = true)
 |-- extra_infos: struct (nullable = true)
 |    |-- Hướng ban công: string (nullable = true)
 |    |-- Hướng nhà: string (nullable = true)
 |    |-- Mặt tiền: string (nullable = true)
 |    |-- Pháp lý: string (nullable = true)
 |    |-- Số phòng: string (nullable = true)
 |    |-- Số phòng khách: string (nullable = true)
 |    |-- Số phòng ngủ: string (nullable = true)
 |    |-- Số phòng ngủ :: string (nullable = true)
 |    |-- Số toilet: string (nullable = true)
 |    |-- Số toilet :: string (nullable = true)
 |    |-- Số tầng: string (nullable = true)


In [16]:
df = df.withColumn('Id', monotonically_increasing_id())


In [17]:
df = df.withColumn("text", concat_ws(' ', col('title'), col('description')))
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
df = tokenizer.transform(df)


In [18]:
hashingTF = HashingTF(inputCol="tokens", outputCol="tf")
df = hashingTF.transform(df)

idf = IDF(inputCol="tf", outputCol="tfidf")
idf_model = idf.fit(df)
df = idf_model.transform(df)

In [19]:
append_non_zero_udf = udf(lambda v: Vectors.sparse(len(v) + 1, list(v.indices) + [len(v)], list(v.values) + [1e-5]), VectorUDT())

df = df.withColumn("tfidf", append_non_zero_udf(col("tfidf")))

In [20]:
num_hash_tables = 5
minhashLSH = MinHashLSH(inputCol="tfidf", outputCol="hashes", numHashTables=num_hash_tables)
model = minhashLSH.fit(df)
df_duplicates = model.approxSimilarityJoin(df.select("Id", "tfidf"), df.select("Id", "tfidf"), 0.8, distCol="JaccardDistance") \
    .filter("datasetA.id < datasetB.id")  # Avoid comparing a row to itself

In [21]:
df_duplicates.show(20)

Py4JJavaError: An error occurred while calling o313.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 9.0 failed 1 times, most recent failure: Lost task 3.0 in stage 9.0 (TID 49) (HauMoon executor driver): java.net.SocketException: Connection reset by peer
	at java.base/sun.nio.ch.NioSocketImpl.implWrite(NioSocketImpl.java:425)
	at java.base/sun.nio.ch.NioSocketImpl.write(NioSocketImpl.java:445)
	at java.base/sun.nio.ch.NioSocketImpl$2.write(NioSocketImpl.java:831)
	at java.base/java.net.Socket$SocketOutputStream.write(Socket.java:1035)
	at java.base/java.io.BufferedOutputStream.write(BufferedOutputStream.java:123)
	at java.base/java.io.DataOutputStream.write(DataOutputStream.java:112)
	at java.base/java.io.FilterOutputStream.write(FilterOutputStream.java:108)
	at org.apache.spark.api.python.PythonRDD$.write$1(PythonRDD.scala:310)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1(PythonRDD.scala:322)
	at